In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from pprint import pprint
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed


categories = [
    "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
    "https://www.hireitpeople.com/resume-database/71-sap-resumes",
    "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
]

categories = [
        "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
]




def scrape_resume_links(category_url, max_pages=200):
    """
    Purpose: scrapes resume page URLs from a given category link that
    contains multiple paged resume listings. eg: https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes

    working logic:
    1. resume_links' to store the extracted URLs.
    2. define HTTP request headers to mimic a real web browser and avoid basic blocking mechanisms.
    3. session object is created for efficiency.
    4. loop runs from page 1 up to 'max_pages'. For each iteration:
         - constructs the page URL by appending '/page/{page}' to the category URL.
         - sends a GET request to that page.
         - if the response code is not 200, it assumes there are no more pages and stops.
         - html is parsed using bs4.
         - looks for resume links using the CSS selector 'table.hit-table h4 a'.
         - extracts the 'href' attribute of each <a> tag and adds it to the list.
         - until no links are found.
         - random delay (from 0.3 to 0.7 seconds) is added to avoid server overload or detection.
    5. returns a list of all collected resume page URLs.
    """
    resume_links = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/116.0 Safari/537.36"
    }

    session = requests.Session()
    session.headers.update(headers)

    for page in range(1, max_pages + 1):
        url = f"{category_url}/page/{page}"
        print(f"[+] Scraping {url}")

        r = session.get(url)
        if r.status_code != 200:
            print(f"[-] Page {page} returned {r.status_code}, stopping.")
            break

        soup = BeautifulSoup(r.text, "html.parser")
        links = [a['href'] for a in soup.select("table.hit-table h4 a")]

        if not links:  # No more resumes
            print(f"[-] No resumes found on page {page}, stopping.")
            break

        resume_links.extend(links)

        # small random delay (just to avoid hammering)
        time.sleep(random.uniform(0.3, 0.7))

    return resume_links

all_links = {}

with ThreadPoolExecutor(max_workers=len(categories)) as executor:
    futures = {executor.submit(scrape_resume_links, cat_url, max_pages=200): cat_url for cat_url in categories}
    for future in as_completed(futures):
        cat_url = futures[future]
        try:
            links = future.result()
            all_links[cat_url] = links
            print(f"[✓] Collected {len(links)} resumes from {cat_url}")
        except Exception as e:
            print(f"[!] Failed to scrape {cat_url}: {e}")

# flatten list
all_resume_links = [link for links in all_links.values() for link in links]


[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/1[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/1

[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/2
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/2
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/3
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/3
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/4
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/4
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/5
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/5
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/6
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle

In [2]:
with open("resume_links.txt", "w") as f:
    f.write("\n".join(all_resume_links))

In [ ]:
with open("resume_links.txt", "r") as f:
    all_resume_links = f.readlines()

In [3]:
def scrape_resume_links(category_url, max_pages=200):
    """
    Purpose: scrapes resume page URLs from a given category link that
    contains multiple paged resume listings. eg: https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes

    working logic:
    1. resume_links' to store the extracted URLs.
    2. define HTTP request headers to mimic a real web browser and avoid basic blocking mechanisms.
    3. session object is created for efficiency.
    4. loop runs from page 1 up to 'max_pages'. For each iteration:
         - constructs the page URL by appending '/page/{page}' to the category URL.
         - sends a GET request to that page.
         - if the response code is not 200, it assumes there are no more pages and stops.
         - html is parsed using bs4.
         - looks for resume links using the CSS selector 'table.hit-table h4 a'.
         - extracts the 'href' attribute of each <a> tag and adds it to the list.
         - until no links are found.
         - random delay (from 0.3 to 0.7 seconds) is added to avoid server overload or detection.
    5. returns a list of all collected resume page URLs.
    """
    resume_links = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/116.0 Safari/537.36"
    }

    session = requests.Session()
    session.headers.update(headers)

    for page in range(1, max_pages + 1):
        url = f"{category_url}/page/{page}"
        print(f"[+] Scraping {url}")

        r = session.get(url)
        if r.status_code != 200:
            print(f"[-] Page {page} returned {r.status_code}, stopping.")
            break

        soup = BeautifulSoup(r.text, "html.parser")
        links = [a['href'] for a in soup.select("table.hit-table h4 a")]

        if not links:  # No more resumes
            print(f"[-] No resumes found on page {page}, stopping.")
            break

        resume_links.extend(links)

        # small random delay (just to avoid hammering)
        time.sleep(random.uniform(0.3, 0.7))

    return resume_links

In [46]:
categories = [
    "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
    "https://www.hireitpeople.com/resume-database/71-sap-resumes",
    "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
]

categories = [
        "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
]

In [ ]:
categories = [
    "https://www.hireitpeople.com/resume-database/66-business-analyst-resumes",
    "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
    "https://www.hireitpeople.com/resume-database/71-sap-resumes",
    "https://www.hireitpeople.com/resume-database/73-datawarehousing-etl-informatica-resumes",
    "https://www.hireitpeople.com/resume-database/74-business-intelligence-business-object-resumes",
    "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
    "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes",
    "https://www.hireitpeople.com/resume-database/81-project-manager-resumes",
    "https://www.hireitpeople.com/resume-database/82-quality-assurance-resumes",
    "https://www.hireitpeople.com/resume-database/87-sql-developers-resumes"
]


# "https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes",
# "https://www.hireitpeople.com/resume-database/64-java-developers-architects-resumes",
# "https://www.hireitpeople.com/resume-database/65-informatica-developers-architects-resumes",
# "https://www.hireitpeople.com/resume-database/66-business-analyst-resumes",
# "https://www.hireitpeople.com/resume-database/67-quality-assurance-qa-resumes",
# "https://www.hireitpeople.com/resume-database/68-network-and-systems-administrators-resumes",
# "https://www.hireitpeople.com/resume-database/69-help-desk-support-resumes",
# "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
# "https://www.hireitpeople.com/resume-database/71-sap-resumes",
# "https://www.hireitpeople.com/resume-database/72-web-developer-resumes",
# "https://www.hireitpeople.com/resume-database/73-datawarehousing-etl-informatica-resumes",
# "https://www.hireitpeople.com/resume-database/74-business-intelligence-business-object-resumes",
# "https://www.hireitpeople.com/resume-database/75-mainframe-resumes",
# "https://www.hireitpeople.com/resume-database/76-network-admin-resumes",
# "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
# "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
# "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes",
# "https://www.hireitpeople.com/resume-database/81-project-manager-resumes",
# "https://www.hireitpeople.com/resume-database/82-quality-assurance-resumes",
# "https://www.hireitpeople.com/resume-database/83-recruiter-resumes",
# "https://www.hireitpeople.com/resume-database/85-sas-resumes",
# "https://www.hireitpeople.com/resume-database/86-sharepoint-resumes",
# "https://www.hireitpeople.com/resume-database/87-sql-developers-resumes",
# "https://www.hireitpeople.com/resume-database/88-technical-writers-resumes",
# "https://www.hireitpeople.com/resume-database/89-websphere-resumes",

num_pages = 500    # each page contains 10 resumes

In [47]:

def scrape_resume_links(category_url, max_pages=200):
    """
    Purpose: scrapes resume page URLs from a given category link that
    contains multiple paged resume listings. eg: https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes

    working logic:
    1. resume_links' to store the extracted URLs.
    2. define HTTP request headers to mimic a real web browser and avoid basic blocking mechanisms.
    3. session object is created for efficiency.
    4. loop runs from page 1 up to 'max_pages'. For each iteration:
         - constructs the page URL by appending '/page/{page}' to the category URL.
         - sends a GET request to that page.
         - if the response code is not 200, it assumes there are no more pages and stops.
         - html is parsed using bs4.
         - looks for resume links using the CSS selector 'table.hit-table h4 a'.
         - extracts the 'href' attribute of each <a> tag and adds it to the list.
         - until no links are found.
         - random delay (from 0.3 to 0.7 seconds) is added to avoid server overload or detection.
    5. returns a list of all collected resume page URLs.
    """
    resume_links = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/116.0 Safari/537.36"
    }

    session = requests.Session()
    session.headers.update(headers)

    for page in range(1, max_pages + 1):
        url = f"{category_url}/page/{page}"
        print(f"[+] Scraping {url}")

        r = session.get(url)
        if r.status_code != 200:
            print(f"[-] Page {page} returned {r.status_code}, stopping.")
            break

        soup = BeautifulSoup(r.text, "html.parser")
        links = [a['href'] for a in soup.select("table.hit-table h4 a")]

        if not links:  # No more resumes
            print(f"[-] No resumes found on page {page}, stopping.")
            break

        resume_links.extend(links)

        # small random delay (just to avoid hammering)
        time.sleep(random.uniform(0.3, 0.7))

    return resume_links

all_links = {}

with ThreadPoolExecutor(max_workers=len(categories)) as executor:
    futures = {executor.submit(scrape_resume_links, cat_url, max_pages=200): cat_url for cat_url in categories}
    for future in as_completed(futures):
        cat_url = futures[future]
        try:
            links = future.result()
            all_links[cat_url] = links
            print(f"[✓] Collected {len(links)} resumes from {cat_url}")
        except Exception as e:
            print(f"[!] Failed to scrape {cat_url}: {e}")

# flatten list
all_resume_links = [link for links in all_links.values() for link in links]

[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/2
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/2
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/3
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/3
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/4
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/4
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/5
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/5
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/6
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle

In [50]:
all_links

{'https://www.hireitpeople.com/resume-database/77-oracle-resumes': ['https://www.hireitpeople.com/resume-database/77-oracle-resumes/628150-oracle-pl-sql-developer-resume-deerfield-il-1',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/628148-oracle-pl-sql-developer-resume-deerfield-il',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627839-oracle-pl-sql-developer-resume-san-antonio-tx-50',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627837-oracle-pl-sql-developer-resume-san-antonio-tx-48',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627789-oracle-database-administrator-resume-dubuque-iowa-2',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627754-oracle-pl-sql-developer-resume-san-antonio-tx-47',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627582-oracle-pl-sql-developer-resume-san-antonio-tx-42',
  'https://www.hireitpeople.com/resume-database/77-oracle-resumes/62758

In [5]:
session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0 ..."})

org_resume_dict = {}

print(f"[+] Scraping {len(all_resume_links)} resumes in parallel...")

with ThreadPoolExecutor(max_workers=12) as executor:  # adjust workers
    futures = {executor.submit(scrape_resume, url, session): url for url in all_resume_links}
    
    # tqdm progress bar with total count
    for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping resumes", unit="resume"):
        url = futures[future]
        try:
            data = future.result()
            org_resume_dict[url] = data
        except Exception as e:
            print(f"[!] Failed {url}: {e}")

print(f"[✔] Completed scraping {len(org_resume_dict)} resumes.")

[+] Scraping 49730 resumes in parallel...


Scraping resumes:   0%|                             | 8/49730 [00:06<8:29:42,  1.63resume/s]

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/627034-lead-oracle-database-administrator-resume-highlands-ranch-co-1: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/626958-account-tech-subject-matter-expert-tech-lead-resume-5: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Scraping resumes:   0%|                            | 40/49730 [00:18<5:59:19,  2.30resume/s]

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/624509-oracle-financial-functional-consultant-resume-chicago-il-1: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


...

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/604485-oracle-developer-resume-albany-ny-4: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Scraping resumes:   0%|                           | 156/49730 [01:03<4:41:35,  2.93resume/s]

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/588260-oracle-fusion-middleware-administrator-resume-sacramento-california-3: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Scraping resumes: 100%|███████████████████████████| 49730/49730 [37:46<00:00, 21.94resume/s]

[✔] Completed scraping 49719 resumes.


In [6]:
with open("resumes.json", "w", encoding="utf-8") as f:
    json.dump(org_resume_dict, f, ensure_ascii=False, indent=4)

In [7]:
df = pd.DataFrame.from_dict(org_resume_dict, orient="index").reset_index()
df.rename(columns={"index": "url"}, inplace=True)

# Convert any dict/list column into a JSON string
for col in df.columns:
    df[col] = df[col].apply(
        lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (dict, list)) else x
    )

# Save to Parquet
df.to_parquet("resumes.parquet", engine="pyarrow", index=False)
print("✅ Saved to resumes.parquet")


✅ Saved to resumes.parquet


In [23]:
# all_links = {}
# for cat_url in categories:
#     links = scrape_resume_links(cat_url, max_pages=1)
#     all_links[cat_url] = links
#     print(f"[✓] Collected {len(links)} resumes from {cat_url}")


# with open("resume_links.txt", "w") as f:
#     for cat, links in all_links.items():
#         f.write(f"# {cat}\n")
#         for link in links:
#             f.write(link + "\n")



# org_resume_dict = {}

# for domain_link in all_links.keys():
#     domain_resume_list = []
#     for resume_link in all_links[domain_link]: 
#         resume_dict = scrape_resume(resume_link)
#         domain_resume_list.append(resume_dict)
#         print(f"[✓] {resume_link}")
#     org_resume_dict[domain_link] = domain_resume_list
#     print(f"DOMAIN COMPLETED: {domain_link}")
    

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from pprint import pprint
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed


categories = [
    "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
    "https://www.hireitpeople.com/resume-database/71-sap-resumes",
    "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
]

categories = [
        "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
]




def scrape_resume_links(category_url, max_pages=200):
    """
    Purpose: scrapes resume page URLs from a given category link that
    contains multiple paged resume listings. eg: https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes

    working logic:
    1. resume_links' to store the extracted URLs.
    2. define HTTP request headers to mimic a real web browser and avoid basic blocking mechanisms.
    3. session object is created for efficiency.
    4. loop runs from page 1 up to 'max_pages'. For each iteration:
         - constructs the page URL by appending '/page/{page}' to the category URL.
         - sends a GET request to that page.
         - if the response code is not 200, it assumes there are no more pages and stops.
         - html is parsed using bs4.
         - looks for resume links using the CSS selector 'table.hit-table h4 a'.
         - extracts the 'href' attribute of each <a> tag and adds it to the list.
         - until no links are found.
         - random delay (from 0.3 to 0.7 seconds) is added to avoid server overload or detection.
    5. returns a list of all collected resume page URLs.
    """
    resume_links = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/116.0 Safari/537.36"
    }

    session = requests.Session()
    session.headers.update(headers)

    for page in range(1, max_pages + 1):
        url = f"{category_url}/page/{page}"
        print(f"[+] Scraping {url}")

        r = session.get(url)
        if r.status_code != 200:
            print(f"[-] Page {page} returned {r.status_code}, stopping.")
            break

        soup = BeautifulSoup(r.text, "html.parser")
        links = [a['href'] for a in soup.select("table.hit-table h4 a")]

        if not links:  # No more resumes
            print(f"[-] No resumes found on page {page}, stopping.")
            break

        resume_links.extend(links)

        # small random delay (just to avoid hammering)
        time.sleep(random.uniform(0.3, 0.7))

    return resume_links

all_links = {}

with ThreadPoolExecutor(max_workers=len(categories)) as executor:
    futures = {executor.submit(scrape_resume_links, cat_url, max_pages=200): cat_url for cat_url in categories}
    for future in as_completed(futures):
        cat_url = futures[future]
        try:
            links = future.result()
            all_links[cat_url] = links
            print(f"[✓] Collected {len(links)} resumes from {cat_url}")
        except Exception as e:
            print(f"[!] Failed to scrape {cat_url}: {e}")

# flatten list
all_resume_links = [link for links in all_links.values() for link in links]


[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/1[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/1

[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/2
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/2
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/3
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/3
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/4
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/4
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/5[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/5

[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/6
[+] Scraping https://www.hireitpeople.com/resume-database/77-or

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
import re
import requests
from typing import List, Optional, Tuple
from pydantic import BaseModel
from tqdm import tqdm


class Experience(BaseModel):
    job_role: str
    responsibilities: List[str]
    environment: Optional[str] = None


class Resume(BaseModel):
    job_role: str
    professional_summary: List[str]
    technical_skills: List[str]
    experiences: List[Experience]


class PostExtractionResult(BaseModel):
    """Result model for post extraction step."""
    job_role: Optional[str]
    structured_content: List[dict]
    full_text: str
    container_text: str
    missing_excerpt: str
    skipped_blocks: List[str]
    warnings: List[str]
    
def normalize_breaks(soup):
    """Convert <br> tags to newline text nodes so .get_text() uses them."""
    for br in soup.find_all("br"):
        br.replace_with("\n")

def clean_whitespace(text):
    lines = [ln.strip() for ln in text.splitlines()]
    while lines and lines[0] == "":
        lines.pop(0)
    while lines and lines[-1] == "":
        lines.pop()
    return "\n".join([re.sub(r'\s+', ' ', ln) for ln in lines])

def extract_job_role(soup):
    media_bodies = soup.find_all("div", class_=re.compile(r"media-body"))
    if media_bodies:
        media_body = media_bodies[0]
        job_title_tag = media_body.find("h3")
        if job_title_tag:
            job_role = job_title_tag.get_text(strip=True)
            if job_role:
                return job_role
    return None

def extract_post_body_safe(
    url: str,
    target_class: Optional[str] = None,
    class_regex: Optional[str] = None,
    allow_fallback: bool = True,
    debug: bool = False,
    min_word_threshold: int = 120,
    retries: int = 3,
) -> PostExtractionResult:
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"}
    
    for attempt in range(retries):
        try:
            with requests.Session() as session:
                session.headers.update(headers)
                resp = session.get(url, timeout=20)
                resp.raise_for_status()
                if len(resp.text) < 1000:
                    raise ValueError("Response too short.")
                break
        except Exception as e:
            if attempt == retries - 1:
                raise
            time.sleep(random.uniform(1, 3))
    soup = BeautifulSoup(resp.text, "html.parser")

    normalize_breaks(soup)

    # Identify container
    container = None
    if target_class:
        container = soup.find("div", class_=target_class)
    if not container and class_regex:
        container = soup.find("div", class_=re.compile(class_regex))
    if not container and allow_fallback:
        divs = soup.find_all("div")
        if divs:
            container = max(divs, key=lambda d: len(d.get_text(strip=True)))
    if not container:
        raise ValueError("Could not find a suitable container.")

    raw_container_text = container.get_text(separator="\n", strip=True)
    container_text = clean_whitespace(raw_container_text)

    structured_content = []
    skipped_blocks = []  # 🔹 Track skipped text blocks

    resume_job_role = extract_job_role(soup)

    # 2️⃣ Handle normal paragraphs and lists not under media-body
    for element in container.find_all(["p", "ul"], recursive=True):
        if element.find_parent("div", class_=re.compile(r"media-body")):
            continue  # already captured above

        if element.name == "p":
            text = clean_whitespace(" ".join(element.stripped_strings))
            # 🚫 Skip overly long text blocks
            if len(text.split()) > min_word_threshold:
                skipped_blocks.append(text[:120] + "...")
                continue
            if text:
                structured_content.append({"type": "p", "text": text})

        elif element.name == "ul":
            items = []
            for li in element.find_all("li", recursive=False):
                li_text = clean_whitespace(" ".join(li.stripped_strings))
                # 🚫 Skip overly long list items
                if len(li_text.split()) > min_word_threshold:
                    skipped_blocks.append(li_text[:120] + "...")
                    continue
                if li_text:
                    items.append(li_text)
            if items:
                structured_content.append({"type": "ul", "items": items})

    # Join all paragraph text for convenience
    joined_p = "\n\n".join(
        [b["text"] for b in structured_content if b.get("type") == "p"]
    )

    container_words = len(container_text.split())
    joined_words = len(joined_p.split()) if joined_p else 0

    warnings = []
    missing_excerpt = ""
    if container_words > joined_words + 20:
        temp = container_text
        for block in structured_content:
            if block.get("type") == "p":
                temp = temp.replace(block["text"], "")
            elif block.get("type") == "ul":
                for item in block["items"]:
                    temp = temp.replace(item, "")
            elif "job_role" in block:
                temp = temp.replace(block["job_role"], "")
                for sub in block.get("content", []):
                    if sub.get("type") == "p":
                        temp = temp.replace(sub["text"], "")
                    elif sub.get("type") == "ul":
                        for item in sub["items"]:
                            temp = temp.replace(item, "")
        missing_excerpt = temp.strip()[:800]
        if missing_excerpt:
            warnings.append("Container has additional text not captured by structured tags.")

    if "<script" in resp.text.lower() and (container_words == 0 or joined_words == 0):
        warnings.append("Page might be JS-rendered.")

    if debug:
        print("===== DEBUG INFO =====")
        print("Container classes:", container.get("class"))
        print("Job roles found:", sum(1 for b in structured_content if "job_role" in b))
        print("Paragraphs:", sum(1 for b in structured_content if b.get("type") == "p"))
        print("Lists:", sum(1 for b in structured_content if b.get("type") == "ul"))
        print("Skipped blocks:", len(skipped_blocks))
        print("Warnings:", warnings)
        print("======================")

    return PostExtractionResult(
        job_role=resume_job_role,
        structured_content=structured_content,
        full_text=joined_p,
        container_text=container_text,
        missing_excerpt=missing_excerpt,
        skipped_blocks=skipped_blocks,
        warnings=warnings,
    )

def parse_resume(json_data):
    # Initialize the resume structure
    resume = {
        "job_role": json_data.get("job_role", ""),
        "professional_summary": [],
        "technical_skills": [],
        "experiences": []
    }
    
    structured_content = json_data.get("structured_content", [])
    
    # Flags to track current section
    in_summary = False
    in_technical_skills = False
    in_professional_experience = False
    
    # Variables for experience parsing
    current_experience = None
    current_job_role = None
    current_responsibilities = []
    current_environment = None
    experience_started = False
    
    # Iterate through each element in structured_content
    i = 0
    while i < len(structured_content):
        element = structured_content[i]
        
        # Check if we're entering SUMMARY section
        if element["type"] == "p" and "SUMMARY" in element["text"]:
            in_summary = True
            in_technical_skills = False
            in_professional_experience = False
            i += 1
            continue
        
        # Check if we're entering TECHNICAL SKILLS section
        elif element["type"] == "p" and "TECHNICAL SKILLS" in element["text"]:
            in_summary = False
            in_technical_skills = True
            in_professional_experience = False
            i += 1
            continue
        
        # Check if we're entering PROFESSIONAL EXPERIENCE section
        elif element["type"] == "p" and "PROFESSIONAL EXPERIENCE" in element["text"]:
            in_summary = False
            in_technical_skills = False
            in_professional_experience = True
            i += 1
            continue
        
        # Process SUMMARY section
        elif in_summary:
            if element["type"] == "ul":
                resume["professional_summary"].extend(element["items"])
            i += 1
            continue
        
        # Process TECHNICAL SKILLS section
        elif in_technical_skills:
            if element["type"] == "p":
                # Skip the "TECHNICAL SKILLS:" header itself
                if "TECHNICAL SKILLS" not in element["text"]:
                    resume["technical_skills"].append(element["text"])
            i += 1
            continue
        
        # Process PROFESSIONAL EXPERIENCE section
        elif in_professional_experience:
            # Check for "Confidential" P tags to identify new experiences
            if element["type"] == "p" and "Confidential" in element["text"]:
                # Save previous experience if it exists and has data
                if experience_started and current_job_role:
                    experience_data = {
                        "job_role": current_job_role,
                        "responsibilities": current_responsibilities.copy()
                    }
                    if current_environment:
                        experience_data["environment"] = current_environment
                    resume["experiences"].append(experience_data)
                
                # Reset for new experience
                current_job_role = None
                current_responsibilities = []
                current_environment = None
                experience_started = True
                
                # The next P tag after "Confidential" should be the job role
                if i + 1 < len(structured_content):
                    next_element = structured_content[i + 1]
                    if next_element["type"] == "p":
                        current_job_role = next_element["text"]
                        i += 2  # Skip both confidential and job role
                    else:
                        i += 1
                else:
                    i += 1
                continue
            
            # Check for UL tags (responsibilities)
            elif element["type"] == "ul" and current_job_role:
                current_responsibilities.extend(element["items"])
                i += 1
                continue
            
            # Check for "Environment" P tags
            elif element["type"] == "p" and "Environment:" in element["text"]:
                current_environment = element["text"].replace("Environment:", "").strip()
                i += 1
                continue
            
            # Check for "Environment" without colon (some entries might have different formatting)
            elif element["type"] == "p" and "Environment" in element["text"] and current_job_role:
                current_environment = element["text"].replace("Environment", "").strip()
                if current_environment.startswith(":"):
                    current_environment = current_environment[1:].strip()
                i += 1
                continue
            
            # Regular P tag in experience section (might be environment or other info)
            elif element["type"] == "p":
                # If we don't have a job role yet but we're in an experience, this might be it
                if not current_job_role and "Confidential" not in element["text"] and "PROFESSIONAL EXPERIENCE" not in element["text"]:
                    current_job_role = element["text"]
                i += 1
                continue
            
            else:
                i += 1
                continue
        
        else:
            i += 1
            continue
    
    # Don't forget to add the last experience if it exists
    if experience_started and current_job_role:
        experience_data = {
            "job_role": current_job_role,
            "responsibilities": current_responsibilities.copy()
        }
        if current_environment:
            experience_data["environment"] = current_environment
        resume["experiences"].append(experience_data)
    
    return resume

import re
import json

def validate_structured_resume(json_data):
    structured_content = json_data.get("structured_content", [])
    errors = []
    i = 0
    n = len(structured_content)
    section_order = []

    def expect_type(index, expected_type, context):
        """Helper: validate type field"""
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        if structured_content[index].get("type") != expected_type:
            errors.append(f"Expected type '{expected_type}' at {context}, got '{structured_content[index].get('type')}'")
            return False
        return True

    def expect_text(index, expected_text, context, case_sensitive=True, startswith=False):
        """Helper: validate text content"""
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        text = structured_content[index].get("text", "")
        cmp_text = text if case_sensitive else text.upper()
        cmp_expected = expected_text if case_sensitive else expected_text.upper()

        if startswith:
            if not cmp_text.startswith(cmp_expected):
                errors.append(f"Expected text starting with '{expected_text}' at {context}, got '{text}'")
                return False
        elif cmp_text != cmp_expected:
            errors.append(f"Expected text '{expected_text}' at {context}, got '{text}'")
            return False
        return True

    # --- 1. OBJECTIVE (optional) ---
    if i < n and structured_content[i].get("text", "").strip().upper() == "OBJECTIVE":
        section_order.append("OBJECTIVE")
        expect_type(i, "p", "OBJECTIVE header")
        i += 1
        if i < n and structured_content[i].get("type") == "ul":
            if not isinstance(structured_content[i].get("items"), list):
                errors.append("OBJECTIVE ul should have a list of items")
            i += 1

    # --- 2. SUMMARY ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "SUMMARY":
        errors.append("Missing SUMMARY section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("SUMMARY")
    expect_type(i, "p", "SUMMARY header")
    i += 1

    expect_type(i, "ul", "SUMMARY bullets")
    if not isinstance(structured_content[i].get("items"), list):
        errors.append("SUMMARY ul must contain list of strings")
    i += 1

    # --- 3. TECHNICAL SKILLS ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "TECHNICAL SKILLS":
        errors.append("Missing TECHNICAL SKILLS section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("TECHNICAL SKILLS")
    expect_type(i, "p", "TECHNICAL SKILLS header")
    i += 1

    while i < n and structured_content[i].get("type") == "p" and structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        expect_type(i, "p", "TECHNICAL SKILL item")
        i += 1

    # --- 4. PROFESSIONAL EXPERIENCE ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        errors.append("Missing PROFESSIONAL EXPERIENCE section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("PROFESSIONAL EXPERIENCE")
    expect_type(i, "p", "PROFESSIONAL EXPERIENCE header")
    i += 1

    # --- 5. Validate experience blocks ---
    while i < n:
        # Company line
        if not (structured_content[i].get("type") == "p" and structured_content[i].get("text", "").startswith("Confidential")):
            errors.append(f"Expected company line starting with 'Confidential' at index {i}")
            break
        i += 1

        # Job title
        expect_type(i, "p", "Job Role")
        i += 1

        # Responsibilities
        if not (structured_content[i].get("type") == "p" and re.match(r"(?i)^responsibilities", structured_content[i].get("text", "").strip())):
            errors.append(f"Expected 'Responsibilities:' line at index {i}")
            break
        i += 1

        # Responsibilities list
        expect_type(i, "ul", "Responsibilities UL")
        if not isinstance(structured_content[i].get("items"), list):
            errors.append(f"Responsibilities UL at index {i} must contain a list of strings")
        i += 1

        # Environment line (optional)
        if i < n and structured_content[i].get("type") == "p" and structured_content[i].get("text", "").startswith("Environment"):
            i += 1

    return {"is_valid": len(errors) == 0, "errors": errors, "sections_found": section_order}

# Alternative version that returns a Resume object
def parse_resume_to_object(url: str) -> Tuple[Optional[Resume], Optional[str]]:
    """
    Returns (Resume object, failed_url_if_any).
    """
    try:
        scraped_data = extract_post_body_safe(
            url,
            class_regex=r"(single-post-body|post-content|entry-content|article-body)",
            allow_fallback=True
        )

        # Validate structure
        valid_check = validate_structured_resume(scraped_data.model_dump())
        if not valid_check["is_valid"]:
            return None, url  # Invalid resume structure

        parsed_data = parse_resume(scraped_data.model_dump())
        resume_obj = Resume(**parsed_data)
        return resume_obj, None  # success

    except Exception as e:
        # Any parsing/scraping failure
        return None, url



# --- Run concurrent scraping and merge results ---
def scrape_and_parse_all(all_resume_links: List[str], max_workers: int = 12):
    org_resume_dict = {}
    failed_urls = []

    print(f"[+] Scraping {len(all_resume_links)} resumes in parallel...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_resume_to_object, url): url for url in all_resume_links}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping resumes", unit="resume"):
            url = futures[future]
            try:
                resume_obj, failed_url = future.result()
                if resume_obj:
                    org_resume_dict[url] = resume_obj.model_dump()  # or keep as object
                if failed_url:
                    failed_urls.append(failed_url)
            except Exception as e:
                print(f"[!] Failed {url}: {e}")
                failed_urls.append(url)

    return org_resume_dict, failed_urls


# --- Example usage ---
# all_resume_links = [...]  # your list of resume URLs
with open("resume_links.txt", "r") as f:
    all_resume_links = f.readlines()

org_resume_dict, failed_urls = scrape_and_parse_all(all_resume_links)



print(f"\n✅ Successfully parsed {len(org_resume_dict)} resumes")
print(f"❌ Failed to parse {len(failed_urls)} resumes")



[+] Scraping 4000 resumes in parallel...


Scraping resumes: 100%|██████████| 4000/4000 [08:19<00:00,  8.01resume/s]  


✅ Successfully parsed 1096 resumes
❌ Failed to parse 2904 resumes


In [58]:
org_resume_dict_2, failed_urls_2 = scrape_and_parse_all(failed_urls)



print(f"\n✅ Successfully parsed {len(org_resume_dict_2)} resumes")
print(f"❌ Failed to parse {len(failed_urls_2)} resumes")

[+] Scraping 2904 resumes in parallel...


Scraping resumes: 100%|██████████| 2904/2904 [05:26<00:00,  8.90resume/s]


✅ Successfully parsed 0 resumes
❌ Failed to parse 2904 resumes


In [59]:
failed_urls_2

['https://www.hireitpeople.com/resume-database/77-oracle-resumes/626615-oracle-pl-sql-developer-resume-san-antonio-tx-33',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/626617-oracle-pl-sql-developer-resume-san-antonio-tx-35',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/625471-oracle-applications-functional-consultant-resume-moosic-pa',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627452-java-oracle-programmer-resume-ca',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627505-java-oracle-programmer-resume-palo-alto',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627267-oracle-dba-resume-minneapolis-mn-10',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627034-lead-oracle-database-administrator-resume-highlands-ranch-co-1',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/626958-account-tech-subject-matter-expert-tech-lead-resume-5',
 'https://www.hireitpeopl

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
import re
import requests
from typing import List, Optional, Tuple
from pydantic import BaseModel
from tqdm import tqdm


class Experience(BaseModel):
    job_role: str
    responsibilities: List[str]
    environment: Optional[str] = None


class Resume(BaseModel):
    job_role: str
    professional_summary: List[str]
    technical_skills: List[str]
    experiences: List[Experience]


class PostExtractionResult(BaseModel):
    """Result model for post extraction step."""
    job_role: Optional[str]
    structured_content: List[dict]
    full_text: str
    container_text: str
    missing_excerpt: str
    skipped_blocks: List[str]
    warnings: List[str]
    
def normalize_breaks(soup):
    """Convert <br> tags to newline text nodes so .get_text() uses them."""
    for br in soup.find_all("br"):
        br.replace_with("\n")

def clean_whitespace(text):
    lines = [ln.strip() for ln in text.splitlines()]
    while lines and lines[0] == "":
        lines.pop(0)
    while lines and lines[-1] == "":
        lines.pop()
    return "\n".join([re.sub(r'\s+', ' ', ln) for ln in lines])

def extract_job_role(soup):
    media_bodies = soup.find_all("div", class_=re.compile(r"media-body"))
    if media_bodies:
        media_body = media_bodies[0]
        job_title_tag = media_body.find("h3")
        if job_title_tag:
            job_role = job_title_tag.get_text(strip=True)
            if job_role:
                return job_role
    return None

def extract_post_body_safe(
    url: str,
    target_class: Optional[str] = None,
    class_regex: Optional[str] = None,
    allow_fallback: bool = True,
    debug: bool = False,
    min_word_threshold: int = 120,
    retries: int = 3,
) -> PostExtractionResult:
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"}
    
    for attempt in range(retries):
        try:
            with requests.Session() as session:
                session.headers.update(headers)
                resp = session.get(url, timeout=20)
                resp.raise_for_status()
                if len(resp.text) < 1000:
                    raise ValueError("Response too short.")
                break
        except Exception as e:
            if attempt == retries - 1:
                raise
            time.sleep(random.uniform(1, 3))
    soup = BeautifulSoup(resp.text, "html.parser")

    normalize_breaks(soup)

    # Identify container
    container = None
    if target_class:
        container = soup.find("div", class_=target_class)
    if not container and class_regex:
        container = soup.find("div", class_=re.compile(class_regex))
    if not container and allow_fallback:
        divs = soup.find_all("div")
        if divs:
            container = max(divs, key=lambda d: len(d.get_text(strip=True)))
    if not container:
        raise ValueError("Could not find a suitable container.")

    raw_container_text = container.get_text(separator="\n", strip=True)
    container_text = clean_whitespace(raw_container_text)

    structured_content = []
    skipped_blocks = []  # 🔹 Track skipped text blocks

    resume_job_role = extract_job_role(soup)

    # 2️⃣ Handle normal paragraphs and lists not under media-body
    for element in container.find_all(["p", "ul"], recursive=True):
        if element.find_parent("div", class_=re.compile(r"media-body")):
            continue  # already captured above

        if element.name == "p":
            text = clean_whitespace(" ".join(element.stripped_strings))
            # 🚫 Skip overly long text blocks
            if len(text.split()) > min_word_threshold:
                skipped_blocks.append(text[:120] + "...")
                continue
            if text:
                structured_content.append({"type": "p", "text": text})

        elif element.name == "ul":
            items = []
            for li in element.find_all("li", recursive=False):
                li_text = clean_whitespace(" ".join(li.stripped_strings))
                # 🚫 Skip overly long list items
                if len(li_text.split()) > min_word_threshold:
                    skipped_blocks.append(li_text[:120] + "...")
                    continue
                if li_text:
                    items.append(li_text)
            if items:
                structured_content.append({"type": "ul", "items": items})

    # Join all paragraph text for convenience
    joined_p = "\n\n".join(
        [b["text"] for b in structured_content if b.get("type") == "p"]
    )

    container_words = len(container_text.split())
    joined_words = len(joined_p.split()) if joined_p else 0

    warnings = []
    missing_excerpt = ""
    if container_words > joined_words + 20:
        temp = container_text
        for block in structured_content:
            if block.get("type") == "p":
                temp = temp.replace(block["text"], "")
            elif block.get("type") == "ul":
                for item in block["items"]:
                    temp = temp.replace(item, "")
            elif "job_role" in block:
                temp = temp.replace(block["job_role"], "")
                for sub in block.get("content", []):
                    if sub.get("type") == "p":
                        temp = temp.replace(sub["text"], "")
                    elif sub.get("type") == "ul":
                        for item in sub["items"]:
                            temp = temp.replace(item, "")
        missing_excerpt = temp.strip()[:800]
        if missing_excerpt:
            warnings.append("Container has additional text not captured by structured tags.")

    if "<script" in resp.text.lower() and (container_words == 0 or joined_words == 0):
        warnings.append("Page might be JS-rendered.")

    if debug:
        print("===== DEBUG INFO =====")
        print("Container classes:", container.get("class"))
        print("Job roles found:", sum(1 for b in structured_content if "job_role" in b))
        print("Paragraphs:", sum(1 for b in structured_content if b.get("type") == "p"))
        print("Lists:", sum(1 for b in structured_content if b.get("type") == "ul"))
        print("Skipped blocks:", len(skipped_blocks))
        print("Warnings:", warnings)
        print("======================")

    return PostExtractionResult(
        job_role=resume_job_role,
        structured_content=structured_content,
        full_text=joined_p,
        container_text=container_text,
        missing_excerpt=missing_excerpt,
        skipped_blocks=skipped_blocks,
        warnings=warnings,
    )

def parse_resume(json_data):
    # Initialize the resume structure
    resume = {
        "job_role": json_data.get("job_role", ""),
        "professional_summary": [],
        "technical_skills": [],
        "experiences": []
    }
    
    structured_content = json_data.get("structured_content", [])
    
    # Flags to track current section
    in_summary = False
    in_technical_skills = False
    in_professional_experience = False
    
    # Variables for experience parsing
    current_experience = None
    current_job_role = None
    current_responsibilities = []
    current_environment = None
    experience_started = False
    
    # Iterate through each element in structured_content
    i = 0
    while i < len(structured_content):
        element = structured_content[i]
        
        # Check if we're entering SUMMARY section
        if element["type"] == "p" and "SUMMARY" in element["text"]:
            in_summary = True
            in_technical_skills = False
            in_professional_experience = False
            i += 1
            continue
        
        # Check if we're entering TECHNICAL SKILLS section
        elif element["type"] == "p" and "TECHNICAL SKILLS" in element["text"]:
            in_summary = False
            in_technical_skills = True
            in_professional_experience = False
            i += 1
            continue
        
        # Check if we're entering PROFESSIONAL EXPERIENCE section
        elif element["type"] == "p" and "PROFESSIONAL EXPERIENCE" in element["text"]:
            in_summary = False
            in_technical_skills = False
            in_professional_experience = True
            i += 1
            continue
        
        # Process SUMMARY section
        elif in_summary:
            if element["type"] == "ul":
                resume["professional_summary"].extend(element["items"])
            i += 1
            continue
        
        # Process TECHNICAL SKILLS section
        elif in_technical_skills:
            if element["type"] == "p":
                # Skip the "TECHNICAL SKILLS:" header itself
                if "TECHNICAL SKILLS" not in element["text"]:
                    resume["technical_skills"].append(element["text"])
            i += 1
            continue
        
        # Process PROFESSIONAL EXPERIENCE section
        elif in_professional_experience:
            # Check for "Confidential" P tags to identify new experiences
            if element["type"] == "p" and "Confidential" in element["text"]:
                # Save previous experience if it exists and has data
                if experience_started and current_job_role:
                    experience_data = {
                        "job_role": current_job_role,
                        "responsibilities": current_responsibilities.copy()
                    }
                    if current_environment:
                        experience_data["environment"] = current_environment
                    resume["experiences"].append(experience_data)
                
                # Reset for new experience
                current_job_role = None
                current_responsibilities = []
                current_environment = None
                experience_started = True
                
                # The next P tag after "Confidential" should be the job role
                if i + 1 < len(structured_content):
                    next_element = structured_content[i + 1]
                    if next_element["type"] == "p":
                        current_job_role = next_element["text"]
                        i += 2  # Skip both confidential and job role
                    else:
                        i += 1
                else:
                    i += 1
                continue
            
            # Check for UL tags (responsibilities)
            elif element["type"] == "ul" and current_job_role:
                current_responsibilities.extend(element["items"])
                i += 1
                continue
            
            # Check for "Environment" P tags
            elif element["type"] == "p" and "Environment:" in element["text"]:
                current_environment = element["text"].replace("Environment:", "").strip()
                i += 1
                continue
            
            # Check for "Environment" without colon (some entries might have different formatting)
            elif element["type"] == "p" and "Environment" in element["text"] and current_job_role:
                current_environment = element["text"].replace("Environment", "").strip()
                if current_environment.startswith(":"):
                    current_environment = current_environment[1:].strip()
                i += 1
                continue
            
            # Regular P tag in experience section (might be environment or other info)
            elif element["type"] == "p":
                # If we don't have a job role yet but we're in an experience, this might be it
                if not current_job_role and "Confidential" not in element["text"] and "PROFESSIONAL EXPERIENCE" not in element["text"]:
                    current_job_role = element["text"]
                i += 1
                continue
            
            else:
                i += 1
                continue
        
        else:
            i += 1
            continue
    
    # Don't forget to add the last experience if it exists
    if experience_started and current_job_role:
        experience_data = {
            "job_role": current_job_role,
            "responsibilities": current_responsibilities.copy()
        }
        if current_environment:
            experience_data["environment"] = current_environment
        resume["experiences"].append(experience_data)
    
    return resume

import re
import json

def validate_structured_resume(json_data):
    structured_content = json_data.get("structured_content", [])
    errors = []
    i = 0
    n = len(structured_content)
    section_order = []

    def expect_type(index, expected_type, context):
        """Helper: validate type field"""
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        if structured_content[index].get("type") != expected_type:
            errors.append(f"Expected type '{expected_type}' at {context}, got '{structured_content[index].get('type')}'")
            return False
        return True

    def expect_text(index, expected_text, context, case_sensitive=True, startswith=False):
        """Helper: validate text content"""
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        text = structured_content[index].get("text", "")
        cmp_text = text if case_sensitive else text.upper()
        cmp_expected = expected_text if case_sensitive else expected_text.upper()

        if startswith:
            if not cmp_text.startswith(cmp_expected):
                errors.append(f"Expected text starting with '{expected_text}' at {context}, got '{text}'")
                return False
        elif cmp_text != cmp_expected:
            errors.append(f"Expected text '{expected_text}' at {context}, got '{text}'")
            return False
        return True

    # --- 1. OBJECTIVE (optional) ---
    if i < n and structured_content[i].get("text", "").strip().upper() == "OBJECTIVE":
        section_order.append("OBJECTIVE")
        expect_type(i, "p", "OBJECTIVE header")
        i += 1
        if i < n and structured_content[i].get("type") == "ul":
            if not isinstance(structured_content[i].get("items"), list):
                errors.append("OBJECTIVE ul should have a list of items")
            i += 1

    # --- 2. SUMMARY ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "SUMMARY":
        errors.append("Missing SUMMARY section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("SUMMARY")
    expect_type(i, "p", "SUMMARY header")
    i += 1

    expect_type(i, "ul", "SUMMARY bullets")
    if not isinstance(structured_content[i].get("items"), list):
        errors.append("SUMMARY ul must contain list of strings")
    i += 1

    # --- 3. TECHNICAL SKILLS ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "TECHNICAL SKILLS":
        errors.append("Missing TECHNICAL SKILLS section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("TECHNICAL SKILLS")
    expect_type(i, "p", "TECHNICAL SKILLS header")
    i += 1

    while i < n and structured_content[i].get("type") == "p" and structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        expect_type(i, "p", "TECHNICAL SKILL item")
        i += 1

    # --- 4. PROFESSIONAL EXPERIENCE ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        errors.append("Missing PROFESSIONAL EXPERIENCE section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("PROFESSIONAL EXPERIENCE")
    expect_type(i, "p", "PROFESSIONAL EXPERIENCE header")
    i += 1

    # --- 5. Validate experience blocks ---
    while i < n:
        # Company line
        if not (structured_content[i].get("type") == "p" and structured_content[i].get("text", "").startswith("Confidential")):
            errors.append(f"Expected company line starting with 'Confidential' at index {i}")
            break
        i += 1

        # Job title
        expect_type(i, "p", "Job Role")
        i += 1

        # Responsibilities
        if not (structured_content[i].get("type") == "p" and re.match(r"(?i)^responsibilities", structured_content[i].get("text", "").strip())):
            errors.append(f"Expected 'Responsibilities:' line at index {i}")
            break
        i += 1

        # Responsibilities list
        expect_type(i, "ul", "Responsibilities UL")
        if not isinstance(structured_content[i].get("items"), list):
            errors.append(f"Responsibilities UL at index {i} must contain a list of strings")
        i += 1

        # Environment line (optional)
        if i < n and structured_content[i].get("type") == "p" and structured_content[i].get("text", "").startswith("Environment"):
            i += 1

    return {"is_valid": len(errors) == 0, "errors": errors, "sections_found": section_order}

# Alternative version that returns a Resume object
def parse_resume_to_object(url: str) -> Tuple[Optional[Resume], Optional[str]]:
    """
    Returns (Resume object, failed_url_if_any).
    """
    try:
        scraped_data = extract_post_body_safe(
            url,
            class_regex=r"(single-post-body|post-content|entry-content|article-body)",
            allow_fallback=True
        )

        # Validate structure
        valid_check = validate_structured_resume(scraped_data.model_dump())
        if not valid_check["is_valid"]:
            return None, url  # Invalid resume structure

        parsed_data = parse_resume(scraped_data.model_dump())
        resume_obj = Resume(**parsed_data)
        return resume_obj, None  # success

    except Exception as e:
        # Any parsing/scraping failure
        return None, url

parsed_data = {}
failed_urls = []
for job in all_links.keys():
    for resume_url in all_links[job]:
        org_resume_dict, failed_urls = scrape_and_parse_all(resume_url)
        if org_resume_dict is not None:
            parsed_data[resume_url] = org_resume_dict
        if failed_urls is not None:
            failed_urls.append(resume_url)


print(f"\n✅ Successfully parsed {len(org_resume_dict)} resumes")
print(f"❌ Failed to parse {len(failed_urls)} resumes")



[+] Sequentially scraping and parsing 115 resumes...


Processing resumes: 100%|██████████| 115/115 [07:46<00:00,  4.05s/resume]


[+] Sequentially scraping and parsing 113 resumes...


Processing resumes: 100%|██████████| 113/113 [07:19<00:00,  3.89s/resume]


[+] Sequentially scraping and parsing 118 resumes...


...

[+] Sequentially scraping and parsing 118 resumes...


Processing resumes: 100%|██████████| 118/118 [07:48<00:00,  3.97s/resume]


[+] Sequentially scraping and parsing 109 resumes...


Processing resumes:  92%|█████████▏| 100/109 [06:42<00:36,  4.03s/resume]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in extract_post_body_safe:72                                                                     │
│                                                                                                  │
│    69 │   │   try:                                                                               │
│    70 │   │   │   with requests.Session() as session:                                            │
│    71 │   │   │   │   session.headers.update(headers)                                            │
│ ❱  72 │   │   │   │   resp = session.get(url, timeout=20)                                        │
│    73 │   │   │   │   resp.raise_for_status()                                                    │
│    74 │   │   │   │   if len(resp.text) < 1000:                                                  │
│    75 │   │   │   │   │   raise ValueError("Response too short.")                                │
│                                                                                                  │
│ /Users/naveenpoliasetty/Downloads/RAG-1/.venv/lib/python3.12/site-packages/requests/sessions.py: │
│ 602 in get                                                                                       │
│                                                                                                  │
│   599 │   │   """                                                                                │
│   600 │   │                                                                                      │
│   601 │   │   kwargs.setdefault("allow_redirects", True)                                         │
│ ❱ 602 │   │   return self.request("GET", url, **kwargs)                                          │
│   603 │                                                                                          │
│   604 │   def options(self, url, **kwargs):                                                      │
│   605 │   │   r"""Sends a OPTIONS request. Returns :class:`Response` object.                     │
│                                                                                                  │
│ /Users/naveenpoliasetty/Downloads/RAG-1/.venv/lib/python3.12/site-packages/requests/sessions.py: │
│ 575 in request                                                                                   │
│                                                                                                  │
│   572 │   │   │   cookies=cookies,                                                               │
│   573 │   │   │   hooks=hooks,                                                                   │
│   574 │   │   )                                                                                  │
│ ❱ 575 │   │   prep = self.prepare_request(req)                                                   │
│   576 │   │                                                                                      │
│   577 │   │   proxies = proxies or {}                                                            │
│   578                                                                                            │
│                                                                                                  │
│ /Users/naveenpoliasetty/Downloads/RAG-1/.venv/lib/python3.12/site-packages/requests/sessions.py: │
│ 484 in prepare_request                                                                           │
│                                                                                                  │
│   481 │   │   │   auth = get_netrc_auth(request.url)                                             │
│   482 │   │                                                                                      │
│   483 │   │   p = PreparedRequest()                                                              │
│ ❱ 484 │   │   p.prepare(                                   

In [ ]:
all

In [55]:
for job in all_links.keys():
    print(job)


https://www.hireitpeople.com/resume-database/77-oracle-resumes
https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes


In [49]:
all_resume_links

['https://www.hireitpeople.com/resume-database/77-oracle-resumes/628150-oracle-pl-sql-developer-resume-deerfield-il-1',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/628148-oracle-pl-sql-developer-resume-deerfield-il',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627839-oracle-pl-sql-developer-resume-san-antonio-tx-50',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627837-oracle-pl-sql-developer-resume-san-antonio-tx-48',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627789-oracle-database-administrator-resume-dubuque-iowa-2',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627754-oracle-pl-sql-developer-resume-san-antonio-tx-47',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627582-oracle-pl-sql-developer-resume-san-antonio-tx-42',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627581-oracle-pl-sql-developer-resume-san-antonio-tx-41',
 'https://www.hireitp

In [33]:
org_resume_dict[url] = data

In [34]:
org_resume_dict

{'https://www.hireitpeople.com/resume-database/77-oracle-resumes/628150-oracle-pl-sql-developer-resume-deerfield-il-1': Resume(job_role='Oracle Pl/sql Developer Resume', professional_summary=['Having 7 years of experience as Oracle PL/SQL developer and implementation of applications using Oracle Applications. Strong experience in Oracle RDBMS, PLSQL Development, Oracle 9i,10g,11g, SQL, PL/SQL, Developer 2000/Forms and Report 10g/6i, Java, HTML, UNIX and Workflow.', 'Experience in Design, Development, Implementation and support as Oracle Developer in Production, Development and Test Environments using Oracle Tools.', 'Experience in using Oracle tools like SQL * Loader, Toad, SQL Navigator, SQL Developer and Data Loader.', 'Developed SQL Loader control programs, interfaces and PL/SQL validation scripts to transfer flat file data into oracle database and Oracle Applications base tables.', 'Experienced in RDBMS design, data modeling, data normalization and SQL tuning using Indexes.', 'Good

In [36]:
import json

valid_data = {url: res.model_dump() for url, res in org_resume_dict.items() if res is not None}
failed_urls = [url for url, res in org_resume_dict.items() if res is None]

with open("structured_resumes.json", "w") as f:
    json.dump(valid_data, f, indent=4)

if failed_urls:
    with open("failed_urls.txt", "w") as f:
        f.write("\n".join(failed_urls))
    print(f"[!] {len(failed_urls)} resumes failed and were logged.")

[!] 2902 resumes failed and were logged.


In [31]:
import json

with open("scraped_resumes.json", "w") as f:
    json.dump({url: res.model_dump() for url, res in org_resume_dict.items()}, f, indent=2)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import json                                                                                  │
│   2                                                                                              │
│   3 with open("scraped_resumes.json", "w") as f:                                                 │
│ ❱ 4 │   json.dump({url: res.model_dump() for url, res in org_resume_dict.items()}, f, indent     │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'model_dump'

In [18]:
len(org_resume_dict.values())

3998

## Validator

In [ ]:
def validate_structured_resume(json_data):
    structured_content = json_data.get("structured_content", [])
    errors = []
    i = 0
    n = len(structured_content)
    section_order = []

    def expect_type(index, expected_type, context):
        """Helper: validate type field"""
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        if structured_content[index].get("type") != expected_type:
            errors.append(f"Expected type '{expected_type}' at {context}, got '{structured_content[index].get('type')}'")
            return False
        return True

    def expect_text(index, expected_text, context, case_sensitive=True, startswith=False):
        """Helper: validate text content"""
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        text = structured_content[index].get("text", "")
        cmp_text = text if case_sensitive else text.upper()
        cmp_expected = expected_text if case_sensitive else expected_text.upper()

        if startswith:
            if not cmp_text.startswith(cmp_expected):
                errors.append(f"Expected text starting with '{expected_text}' at {context}, got '{text}'")
                return False
        elif cmp_text != cmp_expected:
            errors.append(f"Expected text '{expected_text}' at {context}, got '{text}'")
            return False
        return True

    # --- 1. OBJECTIVE (optional) ---
    if i < n and structured_content[i].get("text", "").strip().upper() == "OBJECTIVE":
        section_order.append("OBJECTIVE")
        expect_type(i, "p", "OBJECTIVE header")
        i += 1
        if i < n and structured_content[i].get("type") == "ul":
            if not isinstance(structured_content[i].get("items"), list):
                errors.append("OBJECTIVE ul should have a list of items")
            i += 1

    # --- 2. SUMMARY ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "SUMMARY":
        errors.append("Missing SUMMARY section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("SUMMARY")
    expect_type(i, "p", "SUMMARY header")
    i += 1

    expect_type(i, "ul", "SUMMARY bullets")
    if not isinstance(structured_content[i].get("items"), list):
        errors.append("SUMMARY ul must contain list of strings")
    i += 1

    # --- 3. TECHNICAL SKILLS ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "TECHNICAL SKILLS":
        errors.append("Missing TECHNICAL SKILLS section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("TECHNICAL SKILLS")
    expect_type(i, "p", "TECHNICAL SKILLS header")
    i += 1

    while i < n and structured_content[i].get("type") == "p" and structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        expect_type(i, "p", "TECHNICAL SKILL item")
        i += 1

    # --- 4. PROFESSIONAL EXPERIENCE ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        errors.append("Missing PROFESSIONAL EXPERIENCE section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("PROFESSIONAL EXPERIENCE")
    expect_type(i, "p", "PROFESSIONAL EXPERIENCE header")
    i += 1

    # --- 5. Validate experience blocks ---
    while i < n:
        # Company line
        if not (structured_content[i].get("type") == "p" and structured_content[i].get("text", "").startswith("Confidential")):
            errors.append(f"Expected company line starting with 'Confidential' at index {i}")
            break
        i += 1

        # Job title
        expect_type(i, "p", "Job Role")
        i += 1

        # Responsibilities
        if not (structured_content[i].get("type") == "p" and re.match(r"(?i)^responsibilities[:\s]*", structured_content[i].get("text", "").strip())):
            errors.append(f"Expected 'Responsibilities:' line at index {i}")
            break
        i += 1

        # Responsibilities list
        expect_type(i, "ul", "Responsibilities UL")
        if not isinstance(structured_content[i].get("items"), list):
            errors.append(f"Responsibilities UL at index {i} must contain a list of strings")
        i += 1

        # Environment line (optional)
        if i < n and structured_content[i].get("type") == "p" and re.match(r"(?i)^environment[:\s]", structured_content[i].get("text", "").strip()):
            i += 1


    return {"is_valid": len(errors) == 0, "errors": errors, "sections_found": section_order}

### New architecture

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
import re
import requests
from typing import List, Optional, Tuple
from pydantic import BaseModel
from tqdm import tqdm


class Experience(BaseModel):
    job_role: str
    responsibilities: List[str]
    environment: Optional[str] = None


class Resume(BaseModel):
    job_role: str
    professional_summary: List[str]
    technical_skills: List[str]
    experiences: List[Experience]


class PostExtractionResult(BaseModel):
    """Result model for post extraction step."""
    job_role: Optional[str]
    structured_content: List[dict]
    full_text: str
    container_text: str
    missing_excerpt: str
    skipped_blocks: List[str]
    warnings: List[str]
    
def normalize_breaks(soup):
    """Convert <br> tags to newline text nodes so .get_text() uses them."""
    for br in soup.find_all("br"):
        br.replace_with("\n")

def clean_whitespace(text):
    lines = [ln.strip() for ln in text.splitlines()]
    while lines and lines[0] == "":
        lines.pop(0)
    while lines and lines[-1] == "":
        lines.pop()
    return "\n".join([re.sub(r'\s+', ' ', ln) for ln in lines])

def extract_job_role(soup):
    media_bodies = soup.find_all("div", class_=re.compile(r"media-body"))
    if media_bodies:
        media_body = media_bodies[0]
        job_title_tag = media_body.find("h3")
        if job_title_tag:
            job_role = job_title_tag.get_text(strip=True)
            if job_role:
                return job_role
    return None

def extract_post_body_safe(
    url: str,
    target_class: Optional[str] = None,
    class_regex: Optional[str] = None,
    allow_fallback: bool = True,
    debug: bool = False,
    min_word_threshold: int = 120,
    retries: int = 3,
) -> PostExtractionResult:
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"}
    
    for attempt in range(retries):
        try:
            with requests.Session() as session:
                session.headers.update(headers)
                resp = session.get(url, timeout=20)
                resp.raise_for_status()
                if len(resp.text) < 1000:
                    raise ValueError("Response too short.")
                break
        except Exception as e:
            if attempt == retries - 1:
                raise
            time.sleep(random.uniform(1, 3))
    soup = BeautifulSoup(resp.text, "html.parser")

    normalize_breaks(soup)

    # Identify container
    container = None
    if target_class:
        container = soup.find("div", class_=target_class)
    if not container and class_regex:
        container = soup.find("div", class_=re.compile(class_regex))
    if not container and allow_fallback:
        divs = soup.find_all("div")
        if divs:
            container = max(divs, key=lambda d: len(d.get_text(strip=True)))
    if not container:
        raise ValueError("Could not find a suitable container.")

    raw_container_text = container.get_text(separator="\n", strip=True)
    container_text = clean_whitespace(raw_container_text)

    structured_content = []
    skipped_blocks = []  # 🔹 Track skipped text blocks

    resume_job_role = extract_job_role(soup)

    # 2️⃣ Handle normal paragraphs and lists not under media-body
    for element in container.find_all(["p", "ul"], recursive=True):
        if element.find_parent("div", class_=re.compile(r"media-body")):
            continue  # already captured above

        if element.name == "p":
            text = clean_whitespace(" ".join(element.stripped_strings))
            # 🚫 Skip overly long text blocks
            if len(text.split()) > min_word_threshold:
                skipped_blocks.append(text[:120] + "...")
                continue
            if text:
                structured_content.append({"type": "p", "text": text})

        elif element.name == "ul":
            items = []
            for li in element.find_all("li", recursive=False):
                li_text = clean_whitespace(" ".join(li.stripped_strings))
                # 🚫 Skip overly long list items
                if len(li_text.split()) > min_word_threshold:
                    skipped_blocks.append(li_text[:120] + "...")
                    continue
                if li_text:
                    items.append(li_text)
            if items:
                structured_content.append({"type": "ul", "items": items})

    # Join all paragraph text for convenience
    joined_p = "\n\n".join(
        [b["text"] for b in structured_content if b.get("type") == "p"]
    )

    container_words = len(container_text.split())
    joined_words = len(joined_p.split()) if joined_p else 0

    warnings = []
    missing_excerpt = ""
    if container_words > joined_words + 20:
        temp = container_text
        for block in structured_content:
            if block.get("type") == "p":
                temp = temp.replace(block["text"], "")
            elif block.get("type") == "ul":
                for item in block["items"]:
                    temp = temp.replace(item, "")
            elif "job_role" in block:
                temp = temp.replace(block["job_role"], "")
                for sub in block.get("content", []):
                    if sub.get("type") == "p":
                        temp = temp.replace(sub["text"], "")
                    elif sub.get("type") == "ul":
                        for item in sub["items"]:
                            temp = temp.replace(item, "")
        missing_excerpt = temp.strip()[:800]
        if missing_excerpt:
            warnings.append("Container has additional text not captured by structured tags.")

    if "<script" in resp.text.lower() and (container_words == 0 or joined_words == 0):
        warnings.append("Page might be JS-rendered.")

    if debug:
        print("===== DEBUG INFO =====")
        print("Container classes:", container.get("class"))
        print("Job roles found:", sum(1 for b in structured_content if "job_role" in b))
        print("Paragraphs:", sum(1 for b in structured_content if b.get("type") == "p"))
        print("Lists:", sum(1 for b in structured_content if b.get("type") == "ul"))
        print("Skipped blocks:", len(skipped_blocks))
        print("Warnings:", warnings)
        print("======================")

    return PostExtractionResult(
        job_role=resume_job_role,
        structured_content=structured_content,
        full_text=joined_p,
        container_text=container_text,
        missing_excerpt=missing_excerpt,
        skipped_blocks=skipped_blocks,
        warnings=warnings,
    )

import re

def parse_resume(json_data):
    resume = {
        "job_role": json_data.get("job_role", ""),
        "professional_summary": [],
        "technical_skills": [],
        "experiences": []
    }

    structured_content = json_data.get("structured_content", [])

    # Section flags
    in_summary = False
    in_technical_skills = False
    in_professional_experience = False

    # Experience tracking
    current_experience = None
    current_job_role = None
    current_responsibilities = []
    current_environment = None
    experience_started = False

    i = 0
    while i < len(structured_content):
        element = structured_content[i]
        text = element.get("text", "").strip()

        # ----------------------------
        # SECTION IDENTIFICATION
        # ----------------------------
        upper_text = text.upper()

        if element["type"] == "p" and "SUMMARY" in upper_text:
            in_summary, in_technical_skills, in_professional_experience = True, False, False
            i += 1
            continue

        elif element["type"] == "p" and (
            "TECHNICAL SKILLS" in upper_text or "AREAS OF EXPERIENCE" in upper_text
        ):
            # ④ Alternate label for technical skills handled here
            in_summary, in_technical_skills, in_professional_experience = False, True, False
            i += 1
            continue

        elif element["type"] == "p" and "PROFESSIONAL EXPERIENCE" in upper_text:
            in_summary, in_technical_skills, in_professional_experience = False, False, True
            i += 1
            continue

        # ----------------------------
        # SUMMARY SECTION
        # ----------------------------
        elif in_summary:
            if element["type"] == "ul":
                resume["professional_summary"].extend(element["items"])
            elif element["type"] == "p" and text:
                resume["professional_summary"].append(text)
            i += 1
            continue

        # ----------------------------
        # TECHNICAL SKILLS SECTION
        # ----------------------------
        elif in_technical_skills:
            # ⑤ Handle both <p> and <ul> tags
            if element["type"] == "p" and "TECHNICAL SKILLS" not in upper_text and "AREAS OF EXPERIENCE" not in upper_text:
                resume["technical_skills"].extend([s.strip() for s in re.split(r"[;,\n]", text) if s.strip()])
            elif element["type"] == "ul":
                resume["technical_skills"].extend(element["items"])
            i += 1
            continue

        # ----------------------------
        # PROFESSIONAL EXPERIENCE SECTION
        # ----------------------------
        elif in_professional_experience:
            # ① Handle “Confidential” text merging
            if element["type"] == "p" and "CONFIDENTIAL" in upper_text:
                # Finish the previous experience if valid
                if experience_started and current_job_role and current_responsibilities:
                    exp = {"job_role": current_job_role, "responsibilities": current_responsibilities.copy()}
                    if current_environment:
                        exp["environment"] = current_environment
                    resume["experiences"].append(exp)

                # Reset
                current_job_role = None
                current_responsibilities = []
                current_environment = None
                experience_started = True

                # Separate confidential if merged with text
                if not re.fullmatch(r"Confidential", text, re.IGNORECASE):
                    # e.g. "Confidential - Software Engineer"
                    parts = re.split(r"Confidential\s*[-:]\s*", text, flags=re.IGNORECASE)
                    if len(parts) > 1:
                        current_job_role = parts[1].strip()
                else:
                    # If purely "Confidential", check the next element for job role
                    if i + 1 < len(structured_content):
                        next_el = structured_content[i + 1]
                        if next_el["type"] in ["p", "strong"]:
                            current_job_role = next_el["text"].strip()
                            i += 1  # skip next
                i += 1
                continue

            # ② Handle <strong> tags for job role
            elif element["type"] == "strong":
                current_job_role = text
                i += 1
                continue

            # ③ Skip incomplete subsections (no job role but responsibilities found)
            elif element["type"] == "ul":
                if current_job_role:
                    current_responsibilities.extend(element["items"])
                i += 1
                continue

            # ⑥ Handle environment variations
            elif element["type"] == "p" and re.search(r"(?i)\bEnvironment\b", text):
                current_environment = re.sub(r"(?i)Environment[:\s]*", "", text).strip()
                i += 1
                continue

            # Handle other paragraphs (possibly job role if none yet)
            elif element["type"] == "p" and not current_job_role and "CONFIDENTIAL" not in upper_text:
                current_job_role = text
                i += 1
                continue

            else:
                i += 1
                continue

        else:
            i += 1
            continue

    # ----------------------------
    # ADD LAST EXPERIENCE IF VALID
    # ----------------------------
    if experience_started and current_job_role and current_responsibilities:
        exp = {"job_role": current_job_role, "responsibilities": current_responsibilities.copy()}
        if current_environment:
            exp["environment"] = current_environment
        resume["experiences"].append(exp)

    # ⑦ Filter out unhealthy experience subsections
    resume["experiences"] = [
        exp for exp in resume["experiences"]
        if exp.get("job_role") and exp.get("responsibilities")
    ]

    return resume


import re
import json

import re

def validate_structured_resume(json_data):
    structured_content = json_data.get("structured_content", [])
    errors = []
    i = 0
    n = len(structured_content)
    section_order = []

    def expect_type(index, expected_type, context):
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        if structured_content[index].get("type") != expected_type:
            errors.append(f"Expected type '{expected_type}' at {context}, got '{structured_content[index].get('type')}'")
            return False
        return True

    def expect_text(index, expected_text, context, case_sensitive=False):
        if index >= n:
            errors.append(f"Missing element at {context}")
            return False
        text = structured_content[index].get("text", "")
        if not case_sensitive:
            if text.strip().upper() != expected_text.upper():
                errors.append(f"Expected text '{expected_text}' at {context}, got '{text}'")
                return False
        else:
            if text.strip() != expected_text:
                errors.append(f"Expected text '{expected_text}' at {context}, got '{text}'")
                return False
        return True

    # --- 1. OBJECTIVE (optional) ---
    if i < n and structured_content[i].get("text", "").strip().upper() == "OBJECTIVE":
        section_order.append("OBJECTIVE")
        expect_type(i, "p", "OBJECTIVE header")
        i += 1
        if i < n and structured_content[i].get("type") == "ul":
            if not isinstance(structured_content[i].get("items"), list):
                errors.append("OBJECTIVE ul should have a list of items")
            i += 1

    # --- 2. SUMMARY ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "SUMMARY":
        errors.append("Missing SUMMARY section (must be 'SUMMARY')")
        return {"is_valid": False, "errors": errors}

    section_order.append("SUMMARY")
    expect_type(i, "p", "SUMMARY header")
    i += 1

    expect_type(i, "ul", "SUMMARY bullets")
    if not isinstance(structured_content[i].get("items"), list):
        errors.append("SUMMARY ul must contain list of strings")
    i += 1

    # --- 3. TECHNICAL SKILLS / AREAS OF EXPERIENCE ---
    tech_header_text = structured_content[i].get("text", "").strip().upper() if i < n else ""
    if tech_header_text not in ["TECHNICAL SKILLS", "AREAS OF EXPERIENCE"]:
        errors.append("Missing TECHNICAL SKILLS or AREAS OF EXPERIENCE section (must be all caps)")
        return {"is_valid": False, "errors": errors}

    section_order.append("TECHNICAL SKILLS")
    expect_type(i, "p", "TECHNICAL SKILLS header")
    i += 1

    # Technical skills may appear as paragraphs until PROFESSIONAL EXPERIENCE
    while i < n:
        text = structured_content[i].get("text", "").strip().upper()
        if text == "PROFESSIONAL EXPERIENCE":
            break
        expect_type(i, "p", f"TECHNICAL SKILL item at index {i}")
        i += 1

    # --- 4. PROFESSIONAL EXPERIENCE ---
    if i >= n or structured_content[i].get("text", "").strip().upper() != "PROFESSIONAL EXPERIENCE":
        errors.append("Missing PROFESSIONAL EXPERIENCE section (must be 'PROFESSIONAL EXPERIENCE')")
        return {"is_valid": False, "errors": errors}

    section_order.append("PROFESSIONAL EXPERIENCE")
    expect_type(i, "p", "PROFESSIONAL EXPERIENCE header")
    i += 1

    # --- 5. Validate each experience block ---
    while i < n:
        text_upper = structured_content[i].get("text", "").strip().upper()

        # Company line must start with 'CONFIDENTIAL'
        if not (structured_content[i].get("type") == "p" and text_upper.startswith("CONFIDENTIAL")):
            errors.append(f"Expected 'Confidential' company line at index {i}, got '{structured_content[i].get('text', '')}'")
            break
        i += 1

        # Job title: can be <p> or <strong>
        if i < n:
            if structured_content[i].get("type") not in ["p", "strong"]:
                errors.append(f"Expected job role (<p> or <strong>) after Confidential at index {i}, got '{structured_content[i].get('type')}'")
                break
            i += 1
        else:
            errors.append("Missing job title after Confidential")
            break

        # Responsibilities line (strict)
        if i >= n or not re.match(r"(?i)^responsibilities", structured_content[i].get("text", "").strip()):
            errors.append(f"Expected 'Responsibilities:' line at index {i}")
            break
        expect_type(i, "p", "Responsibilities label")
        i += 1

        # Responsibilities list
        expect_type(i, "ul", "Responsibilities UL")
        if not isinstance(structured_content[i].get("items"), list):
            errors.append(f"Responsibilities UL at index {i} must contain a list of strings")
        i += 1

        # Optional Environment
        if i < n and structured_content[i].get("type") == "p" and re.match(r"(?i)^environment", structured_content[i].get("text", "").strip()):
            i += 1

    return {"is_valid": len(errors) == 0, "errors": errors, "sections_found": section_order}

# Alternative version that returns a Resume object
def parse_resume_to_object(url: str) -> Tuple[Optional[Resume], Optional[str]]:
    """
    Returns (Resume object, failed_url_if_any).
    """
    try:
        scraped_data = extract_post_body_safe(
            url,
            class_regex=r"(single-post-body|post-content|entry-content|article-body)",
            allow_fallback=True
        )

        # Validate structure
        valid_check = validate_structured_resume(scraped_data.model_dump())
        if not valid_check["is_valid"]:
            return None, url  # Invalid resume structure

        parsed_data = parse_resume(scraped_data.model_dump())
        resume_obj = Resume(**parsed_data)
        return resume_obj, None  # success

    except Exception as e:
        # Any parsing/scraping failure
        return None, url

def scrape_and_parse_all(all_resume_links: List[str], max_workers: int = 12):
    org_resume_dict = {}
    failed_urls = []

    print(f"[+] Scraping {len(all_resume_links)} resumes in parallel...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_resume_to_object, url): url for url in all_resume_links}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping resumes", unit="resume"):
            url = futures[future]
            try:
                resume_obj, failed_url = future.result()
                if resume_obj:
                    org_resume_dict[url] = resume_obj.model_dump()  # or keep as object
                if failed_url:
                    failed_urls.append(failed_url)
                    print(f"Failed to parse {failed_url}, total failed: {len(failed_urls)}")
            except Exception as e:
                print(f"[!] Failed {url}: {e}")
                failed_urls.append(url)

    return org_resume_dict, failed_urls


# --- Example usage ---
# all_resume_links = [...]  # your list of resume URLs

org_resume_dict, failed_urls = scrape_and_parse_all(all_resume_links)



print(f"\n✅ Successfully parsed {len(org_resume_dict)} resumes")
print(f"❌ Failed to parse {len(failed_urls)} resumes")



print(f"\n✅ Successfully parsed {len(org_resume_dict)} resumes")
print(f"❌ Failed to parse {len(failed_urls)} resumes")



[+] Scraping 4000 resumes in parallel...


Scraping resumes:   0%|          | 2/4000 [00:02<57:34,  1.16resume/s]  

Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/628084-oracle-soa-bpel-consultant-resume-wilmington-delaware-1, total failed: 1


Scraping resumes:   0%|          | 20/4000 [00:03<07:09,  9.27resume/s]

Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/627269-oracle-dba-resume-ohio-1, total failed: 2
Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/627033-lead-oracle-database-administrator-resume-highlands-ranch-co, total failed: 3
Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/627035-lead-oracle-database-administrator-resume-highlands-ranch-co-2, total failed: 4


...

Scraping resumes:  18%|█▊        | 721/4000 [01:27<07:54,  6.90resume/s]

Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/574254-oracle-apps-dba-resume-sunnyvale-ca-1, total failed: 362


Scraping resumes:  18%|█▊        | 722/4000 [01:27<11:02,  4.95resume/s]

Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/574198-team-leader-resume-nj-1, total failed: 363
Failed to parse https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/574197-team-leader-resume-nj, total failed: 364


Scraping resumes:  18%|█▊        | 733/4000 [01:29<06:37,  8.22resume/s]


In [3]:
all_resume_links

['https://www.hireitpeople.com/resume-database/77-oracle-resumes/628150-oracle-pl-sql-developer-resume-deerfield-il-1',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/628148-oracle-pl-sql-developer-resume-deerfield-il',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627839-oracle-pl-sql-developer-resume-san-antonio-tx-50',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627837-oracle-pl-sql-developer-resume-san-antonio-tx-48',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627789-oracle-database-administrator-resume-dubuque-iowa-2',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627754-oracle-pl-sql-developer-resume-san-antonio-tx-47',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627582-oracle-pl-sql-developer-resume-san-antonio-tx-42',
 'https://www.hireitpeople.com/resume-database/77-oracle-resumes/627581-oracle-pl-sql-developer-resume-san-antonio-tx-41',
 'https://www.hireitp